# Quadratische Diskriminantenanalyse

In dieser Übung werden Sie selbst eine quadratische Diskriminantenanalyse (QDA) implementieren. Zur Erinnerung: Die QDA berechnet $p(x|y)=\frac{p(y|x)*p(x)}{p(y)}$. Die Likelihood $p(y|x)$ wird als normalverteilt angenommen.

## Aufgabe 1
Eine Fischerin benötigt Ihre Hilfe bei der Klassifikation von Fischen. Vor kurzem hat sie folgende Fische gefangen: 

| Länge (m)    | Art          | 
| ------------- |-------------  |
| 1.3           | Barsch       |
| 0.7           | Lachs       |
| 0.62           | Lachs      |
| 0.9           | Lachs       |
| 0.91          | Barsch       |
| 0.31          | Hering       |
| 0.26           | Hering       |

* Berechnen Sie die Priors $p(\omega)$ für jede Fischart
* Wie lautet die Formel für die Berechnung der Parameter $\mu$ und $\sigma^2$ für die Likelihoods?
* Berechnen Sie die Parameter $\mu$ und $\sigma^2$ für die Lkelihoods $p(x|\omega)$. 
* Die Fischerin fängt einen neuen Fisch mit Länge $x = 0.82 m$. Berechen Sie die Posterior-Wahrscheinlichkeit $p(\omega|x)$ für jede Klasse. Wie wird der Fisch klassifiziert?

## Aufgabe 2
Implementieren Sie eine Funktion `priors(classes)`, die für einen Vektor von Klassen-Labels den Prior $p(x)$ für jede Klasse ausgibt.
Die Eingabe soll ein Array von Klassen sein (z.b. `np.array(["stand","sit","sit","stand"])`). Die Ausgabe soll ein Data Frame mit den Spalten `class` und `prior` sein.

In [2]:
import numpy as np
import pandas as pd

def priors(classes):
    unique, counts = np.unique(classes, return_counts=True)
    counts = counts/counts.sum()
    df = pd.DataFrame(np.array([unique,counts]).transpose(),columns=["class","prior"])
    return(df)
    
pp = priors(np.array(["stand","sit","sit","sit","stand"]))
print(pp)
np.array(pp["class"])

   class prior
0    sit   0.6
1  stand   0.4


array(['sit', 'stand'], dtype=object)

## Aufgabe 3
Implementieren Sie eine Funktion `likelihood(data)`, die für ein Data Frame, bestehend aus einer Spalte $y$ und einer Spalte $x$, die Likelihood $p(y|x)$ für jede Klasse $x$ mit einer Normalverteilung approximiert, d.h. es soll für jede Klasse ein Mittelwert und eine Varianz ausgegeben werden.
Die Ausgabe soll also die Spalten `class`, `mean` und `variance` besitzen.


In [3]:
from scipy.io import arff

def likelihood(data):
    uc = np.unique(data["class"]) # gebe mir alle Klassen
    def getMV(c):
        dsub = data.loc[data["class"]==c,"x"] # gebe mir alle x mit Klasse = c
        return([dsub.mean(),dsub.var()]) # berechne mean und var für alle x mit Klasse = c
    mvs = list(map(getMV, uc)) # Für jede Klasse: gebe mean & var
    r = pd.DataFrame(mvs,columns = ["mean","variance"]) # erstelle DF mit Daten von mean & var für jede Klasse
    r["class"] = uc # Füge Spalte "Klasse" hinzu
    return r
    
data = arff.loadarff('features1.arff')
df = pd.DataFrame(data[0])

dat = df.loc[:, ["AccX_mean","class"]]
dat.columns = ["x","class"]
lik = likelihood(dat)
lik

,mean,variance,class
0,523.545587,15.843081,b'bike'
1,517.316002,18.523919,b'downstairs'
2,467.870175,56.697558,b'lie'
3,527.009294,96.483175,b'run'
4,486.721946,71.790221,b'sit'
5,528.594727,1.057665,b'stand'
6,545.413484,62.520043,b'upstairs'
7,525.170962,1.221023,b'walk'


## Aufgabe 4
Implementieren Sie eine Funktion mylda(newdat,lik,priors), die für eine neue Beobachtung `newdat` die wahrscheinlichste Klasse zurückgibt.

Testen Sie Ihre Implementierung auf dem Datensatz `features1.arff`. „Trainieren“ Sie die QDA (d.h. berechnen Sie likelihood und prior), und führen Sie dann für die gleichen Daten eine Klassifikation durch. Wie gut ist die Klassifikation?

In [4]:
from scipy.io import arff
import scipy.stats

def mylda(newdat,lik,prior):
    #Berechne für jeden Datenpunkt: Wahrscheinlichkeit pro Klasse als prior*likelihood, waehle wahrscheinlichste
    def getClass(d):
        def getProb(c,d): # mache alles, was wir oben gemacht haben
            pr = prior.loc[prior["class"]==c,"prior"].values[0] #prior
            m = lik.loc[lik["class"] == c,"mean"].values[0] # mean
            v = lik.loc[lik["class"] == c,"variance"].values[0] #variance
            li = scipy.stats.norm(m, v).pdf(d) #likelihood
            return(li*pr) # Wahrscheinlichkeit
        probs = np.array([getProb(c,d) for c in prior["class"].values]) # mache dies für jeden einzelnen EIntrag für jede Klasse
        return prior.loc[np.argmax(probs),"class"] # Gebe Klasse mit höchster Wahrscheinlichkeit zurück
    newclasses = np.array([getClass(dat) for dat in newdat]) # Array, in dem Klassifikation(nur die Klasse) gespeichert wird
    return newclasses


data = arff.loadarff('features1.arff')
df = pd.DataFrame(data[0])

dat = df.loc[:, ["AccX_mean","class"]]
dat.columns = ["x","class"]

lik = likelihood(dat)
prior = priors(dat["class"])

nc = mylda(dat["x"][1:100],lik,prior) # Klassifiziere mit QDA
print(sum(nc == dat["class"][1:100])/100) # Vergleiche QDA mit korrekten Klassen. Wie viel Übereinstimmung?

0.47
